# Installations and imports

In [1]:
!pip install accelerate
!pip install bitsandbytes

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [4]:
!pip install datasets
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [8]:
import re
import pandas as pd

# Model and dataset installation

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = "codellama/CodeLlama-7b-Instruct-hf"
model = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model, torch_dtype=torch.float16, device_map='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [5]:
dataset = load_dataset("mbpp")
dataset = dataset['test']
dataset

Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
    num_rows: 500
})

# Generating

In [50]:
def make_prompt_list(dataset, shots, num_tests):
  prompt_list = []
  pattern = r'(?<=assert\s)\w+\s*\('
  for i in range(num_tests):
    s = dataset['test_list'][i][0]
    func_name = re.search(pattern, s)
    func_name = func_name.group().strip(' (') if func_name else ''
    prompt = dataset['text'][i] + ' The function should have the following name: ' + func_name + '.\n'
    if shots > 0:
      prompt += 'The code should also pass these tests: '
      for j in range(shots):
         prompt +=  dataset['test_list'][i][j] + ', '
      prompt = prompt[:len(prompt) - 2]
    prompt_list.append(prompt)
  return prompt_list

def generate(dataset, model, shots=0, num_tests=len(dataset)):
  prompt_list = make_prompt_list(dataset, shots, num_tests)
  model.eval()
  responses = []

  for prompt in prompt_list:
      inputs = tokenizer(prompt, return_tensors="pt").to(device)

      with torch.no_grad():
          outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=400)
          response = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

      responses.append(response)
  return prompt_list, responses

Запросы имеют следующий вид в зависимости от параметра shots:

In [42]:
prompt_list = make_prompt_list(dataset, 0, 5)
print(prompt_list[0])
prompt_list = make_prompt_list(dataset, 3, 5)
print(prompt_list[0])

0
Write a python function to remove first and last occurrence of a given character from the string. The function should have the following name: remove_Occ.

3
Write a python function to remove first and last occurrence of a given character from the string. The function should have the following name: remove_Occ.
The code should also pass these tests: assert remove_Occ("hello","l") == "heo", assert remove_Occ("abcda","a") == "bcd", assert remove_Occ("PHP","P") == "H"


In [95]:
NUM_TESTS = 50

In [96]:
prompt_list, responses = generate(dataset, model, 0, NUM_TESTS)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generati

# Testing

In [99]:
def extract_function(text):
  function_lines = []
  inside_function = False

  for line in text.split('\n'):
    if line.startswith('import'):
      function_lines.append(line)
      inside_function = True
    elif line.startswith('def') and inside_function == False:
      function_lines.append(line)
      inside_function = True
    elif inside_function:
      if line == '```' or line == "":
        function_code = '\n'.join(function_lines)
        return function_code
      else:
        function_lines.append(line)

In [ ]:
score = 0
codes = []
tests = []
results = []
for i in range(NUM_TESTS):
  code = extract_function(responses[i])
  codes.append(code)
  test = dataset['test_list'][i][0] + '\n' + dataset['test_list'][i][1] + '\n' + dataset['test_list'][i][2]
  tests.append(test)
  print(i, code)
  code = code + '\n' + test if code else test
  code = code.strip()
  flag = True
  try:
    exec(code)
  except (AssertionError, TypeError, IndentationError, NameError):
    flag = False
    pass
  else:
    score += 1
  results.append('Ok' if flag else 'Error')

# Saving results

In [106]:
df = pd.DataFrame(columns = ['prompt', 'response', 'code', 'tests', 'result'])
for i in range(NUM_TESTS):
  df.loc[i, 'prompt'] = prompt_list[i]
  df.loc[i, 'response'] = responses[i]
  df.loc[i, 'code'] = codes[i]
  df.loc[i, 'tests'] = tests[i]
  df.loc[i, 'result'] = results[i]
df.to_csv('data.csv', index=False)